In [69]:
import json
from IPython.display import Markdown, display

In [84]:
# Load files

with open('ar-en-short_pyannote.json', 'r') as f:
    pyannote_data = json.load(f)

with open('whisper_en.json', 'r') as f:
    whisper_en_data = json.load(f)

with open('whisper_ar.json', 'r') as f:
    whisper_ar_data = json.load(f)

In [87]:
merged_data = []
previous_turn = None

for turn in pyannote_data:
    speaker_id = turn['speaker_id']
    onset = turn['onset']
    duration = turn['duration']

    if previous_turn and previous_turn['speaker_id'] == speaker_id:
        # If the current speaker is the same as the previous speaker,
        # merge the current turn into the previous one by extending the end time.
        previous_turn['duration'] = onset + duration - previous_turn['onset']
    else:
        # If the current speaker is different from the previous speaker,
        # add the previous turn to the merged_data list and set the current turn as the new previous_turn.
        if previous_turn:
            merged_data.append(previous_turn)
        previous_turn = {'speaker_id': speaker_id, 'onset': onset, 'duration': duration}

# Add the last turn to the merged_data list
if previous_turn:
    merged_data.append(previous_turn)

# Print or use the merged_data list as needed
print(merged_data)

[{'speaker_id': 'SPEAKER_02', 'onset': 2.963, 'duration': 15.806999999999999}, {'speaker_id': 'SPEAKER_01', 'onset': 19.601, 'duration': 4.652}, {'speaker_id': 'SPEAKER_00', 'onset': 25.34, 'duration': 17.844}, {'speaker_id': 'SPEAKER_01', 'onset': 44.304, 'duration': 26.519000000000005}, {'speaker_id': 'SPEAKER_02', 'onset': 72.148, 'duration': 6.620999999999995}, {'speaker_id': 'SPEAKER_01', 'onset': 79.346, 'duration': 5.195}, {'speaker_id': 'SPEAKER_00', 'onset': 85.696, 'duration': 41.155}, {'speaker_id': 'SPEAKER_01', 'onset': 128.175, 'duration': 51.22300000000001}, {'speaker_id': 'SPEAKER_02', 'onset': 180.823, 'duration': 15.688}, {'speaker_id': 'SPEAKER_01', 'onset': 197.105, 'duration': 22.496}, {'speaker_id': 'SPEAKER_00', 'onset': 220.45, 'duration': 28.420999999999992}, {'speaker_id': 'SPEAKER_01', 'onset': 249.363, 'duration': 59.23599999999999}, {'speaker_id': 'SPEAKER_00', 'onset': 309.584, 'duration': 8.098}, {'speaker_id': 'SPEAKER_01', 'onset': 309.652, 'duration': 

In [92]:
# Iterate over the Pyannote speaker turns
timeline = []

for turn in merged_data:
    start_time = turn['onset']
    end_time = turn['onset'] + turn['duration']
    speaker_id = turn['speaker_id']

    # Create lists to store the English and Arabic words for the current turn
    en_words = []
    ar_words = []

    # Find the English words that fall within the current speaker turn
    for word in whisper_en_data['words']:
        if word['start'] >= start_time and word['end'] <= end_time:
            en_words.append(word)

    # Find the Arabic words that fall within the current speaker turn
    for word in whisper_ar_data['words']:
        if word['start'] >= start_time and word['end'] <= end_time:
            ar_words.append(word)

    # Create separate segments for English and Arabic if both are present
    if en_words and ar_words:
        # Create an English segment
        en_start_time = min(word['start'] for word in en_words)
        en_end_time = max(word['end'] for word in en_words)
        en_segment = {
            'start_time': en_start_time,
            'end_time': en_end_time,
            'speaker_id': speaker_id,
            'language': 'en',
            'words': en_words,
            'overlapping': True
        }
        timeline.append(en_segment)

        # Create an Arabic segment
        ar_start_time = min(word['start'] for word in ar_words)
        ar_end_time = max(word['end'] for word in ar_words)
        ar_segment = {
            'start_time': ar_start_time,
            'end_time': ar_end_time,
            'speaker_id': speaker_id,
            'language': 'ar',
            'words': ar_words,
            'overlapping': True
        }
        timeline.append(ar_segment)
    else:
        # Create a single segment for the turn
        language = 'en' if en_words else 'ar'
        words = en_words if en_words else ar_words
        segment = {
            'start_time': start_time,
            'end_time': end_time,
            'speaker_id': speaker_id,
            'language': language,
            'words': words,
            'overlapping': False
        }
        timeline.append(segment)

In [93]:
# Print the resulting timeline
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

for turn in timeline:
    if len(turn['words']) == 0:
        continue
    printmd("**" + f"Speaker {turn['speaker_id'][-1]}, Start: {turn['start_time']}, End: {turn['end_time']}" + "**")
    toprint = ("  " + " ".join(word['word'] 
        for word in turn['words']))
    if turn['overlapping'] == True:
        printmd(toprint, color="red")
    else:
        printmd(toprint)
    print()

<span style='color:None'>**Speaker 2, Start: 2.963, End: 18.77**</span>

<span style='color:None'>  Okay so then if we can start again my question next is so what does the economic office in particular have to offer to women in this area</span>

<span style='color:None'>**Speaker 1, Start: 19.601, End: 24.253**</span>

<span style='color:None'>  يقدر لجنة الاقتصاد تقدم لنساء هذه</span>

<span style='color:None'>**Speaker 0, Start: 25.3799991607666, End: 35.91999816894531**</span>

...

<span style='color:red'>  نحب تعوض الأمر بشكل ما leep في م gros لنöhة وفق ان اك animate ان يقدم</span>

<span style='color:None'>**Speaker 2, Start: 387.1600036621094, End: 390.9599914550781**</span>

<span style='color:red'>  obviously as people understand that you're capable of doing this</span>

<span style='color:None'>**Speaker 2, Start: 386.760009765625, End: 389.3800048828125**</span>

<span style='color:red'>  كما تفهم الناس</span>